In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

from siuba import *
import pandas as pd
import numpy as np
import geopandas as gpd

import datetime as dt

import importlib
import rt_analysis as rt
import rt_filter_map_plot

import gcsfs
fs = gcsfs.GCSFileSystem()

import tqdm

# Test Util for Checking/Generating Intermediate Operator Day Analysis Output

In [2]:
# testing parameters: a date that we don't have (saturday)
analysis_date = dt.date(2022, 10, 1)

In [3]:
fs_list = fs.ls(f'{shared_utils.rt_utils.GCS_FILE_PATH}rt_trips/')

In [11]:
def get_operators(analysis_date, operator_list, generate_new=False):
    
    day = str(analysis_date.day).zfill(2)
    month = str(analysis_date.month).zfill(2)
    ## now finds ran operators on specific analysis date
    ran_operators = [int(path.split('rt_trips/')[1].split('_')[0])
                     for path in fs_list
                     if path.split('rt_trips/')[1] and path.split('rt_trips/')[1].split('_')[1] == month and path.split('rt_trips/')[1].split('_')[2][:2] == day]
    
    op_list_runstatus = {}
    for itp_id in operator_list:
        if itp_id in ran_operators:
            print(f'already ran: {itp_id}')
            op_list_runstatus[itp_id] = 'already_ran'
            continue
        else:
            if not generate_new:
                print (f'not yet run: {itp_id}')
                op_list_runstatus[itp_id] = 'not_yet_run'
            elif generate_new:
                print(f'calculating for agency: {itp_id}...')
                try:
                    rt_day = rt.OperatorDayAnalysis(itp_id, analysis_date)
                    rt_day.export_views_gcs()
                    print(f'complete for agency: {itp_id}')
                    op_list_runstatus[itp_id] = 'newly_run'
                except Exception as e:
                    print(f'rt failed for agency {itp_id}')
                    op_list_runstatus[itp_id] = 'new_failed_run'
                    print(e)
    return op_list_runstatus 

In [7]:
get_operators(analysis_date,[293],generate_new=False)

already ran: 293


In [12]:
func_results_2 = get_operators(analysis_date,[208,293],generate_new=True)
print(func_results_2)

calculating for agency: 208...
found parquet
found parquet
found parquet
found parquet
found parquet
complete for agency: 208
already ran: 293
{208: 'newly_run', 293: 'already_ran'}
